In [ ]:
### run once to install nltk and download stopwords
# !pip3 install nltk
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [ ]:
import numpy as np
import pandas as pd
import datetime
import re
import json

In [ ]:
reviews = pd.read_json("processed_data/yelp_team7_dataset_review.json")

### Number of Characters per Review (includes punctuation)

In [ ]:
review_chars = [len(list(x)) for x in reviews.text]

In [ ]:
review_chars_fig =px.histogram(review_chars,
                          title = "Number of characters in reviews",
                         labels = {'value': 'Number of characters'})
review_chars_fig.show()

In [ ]:
reviews["text_char_len"] = review_chars
review_len_x_star_fig = px.box(reviews, 
                                  x = "stars", 
                                  y = "text_char_len", 
                                  title = "Length of Review by Stars")
review_len_x_star_fig.show()

### Number of words per review

In [ ]:
## pull review text into its own list to make the code easier to read
review_text = reviews.text.tolist()
## before taking out ALL punctuation, let's take out apostrophes and leave the contraction combined
review_text = [re.sub("'", "", review).strip() for review in review_text]
review_text_no_punc = [re.sub("[^a-zA-Z]", " ", review).strip() for review in review_text]

In [ ]:
### Let's see if that worked
review_text_no_punc[np.random.randint(0, len(review_text_no_punc))]

In [ ]:
words_per_review = [len(text.split()) for text in review_text_no_punc]
words_per_review_fig =px.histogram(words_per_review,
                          title = "Number of words in reviews",
                         labels = {'value': 'Number of characters'})
words_per_review_fig.show()

In [ ]:
### What are the most common words overall?
# tokens = [word_tokenize(words) for words in review_text_no_punc]
tokens = defaultdict(list)

for i in range(len(review_text_no_punc)):
    for word in review_text_no_punc[i].lower().split():
        tokens[word].append(reviews.review_id[i])


In [ ]:
### export my initial dictionary
with open('processed_data/review_all_tokens.json', "w") as f:
    json.dump(tokens, f)

In [ ]:
tks = tokens.keys()
freqs = []
for key in tks:
    freqs.append(len(tokens[key]))
word_freqs = {"word":tks,
             "frequency": freqs}
word_freqs = pd.DataFrame(word_freqs)
word_freqs.to_csv("processed_data/all_reviews_word_freqs.csv", index = False)

In [ ]:
word_freqs = word_freqs.sort_values(by = "frequency", ascending = False)
word_freqs.head(50)

In [ ]:
nums = ["one", "two", "three", "four", "five"]

for i in range(5):
    print("working on " + str(i+1) + " star reviews")
    review_ids = set(reviews[reviews.stars == i+1].review_id)
    tmp_word_freqs = []
    for key in tks:
        union = review_ids & set(tokens[key])
        tmp_word_freqs.append(len(union)) 
    ret = {"word":tks,
             "frequency": tmp_word_freqs}
    ret = pd.DataFrame(ret)
    flname = "processed_data/" + nums[i] + "_star_review_word_freqs.csv"
    ret.to_csv(flname, index = False)


In [ ]:
## look at pandemic reviews separately?
# pandemic_reviews = reviews[reviews.date >= datetime.datetime(2020, 2, 1)] 
# pandemic_reviews.shape